In [1]:
import numpy as np
import matplotlib.pyplot as plot

In [ ]:
# Read from graph.txt
PLOT_EDGES = True

mode = -1
level = -1
nodes_by_level = []
connections_by_level = []
with open("runs/graph.txt", "r") as f:
    for line in f.readlines():
        line = line.strip()
        if line == "Nodes":
            mode = 0
            level = -1
            continue
        elif line == "Edges" and PLOT_EDGES:
            mode = 1
            level = -1
            continue
        
        if mode == 0:
            if line.startswith("Level"):
                nodes_by_level.append({})
                level = int(line.split(" ")[1])
            else:
                nodes = [float(x) for x in line.split(":")[1].split(",")]
                node_id = int(line.split(":")[0])
                nodes_by_level[level][node_id] = nodes
        elif mode == 1:
            if line.startswith("Level"):
                connections_by_level.append({})
                level = int(line.split(" ")[1])
            else:
                if line[-1] == ",":
                    line = line[:-1]
                connections = [int(x) for x in line.split(":")[1].split(",")]
                node_id = int(line.split(":")[0])
                connections_by_level[level][node_id] = connections

# Plot nodes as 2D points by level
for i in range(len(nodes_by_level)):
    layer_nodes = nodes_by_level[i]
    nodes = np.array(list(layer_nodes.values()))
    plot.title("Level " + str(i))
    plot.scatter(nodes[:, 0], nodes[:, 1])

    if i == 0:
        min_x, max_x = plot.gca().get_xlim()
        min_y, max_y = plot.gca().get_ylim()
    else:
        plot.xlim(min_x, max_x)
        plot.ylim(min_y, max_y)

    # Plot edges
    connections = connections_by_level[i]
    for node_id in connections:
        for connection in connections[node_id]:
            width, height = max_x - min_x, max_y - min_y
            plot.arrow(layer_nodes[node_id][0], layer_nodes[node_id][1], layer_nodes[connection][0] - layer_nodes[node_id][0], layer_nodes[connection][1] - layer_nodes[node_id][1],
                       color='green', alpha=0.3, head_width=width / 35, head_length=height / 35, length_includes_head=True)

    # Export to png
    plot.savefig("runs/Level " + str(i) + ".png")
    plot.show()